In [1]:
import geopandas as gpd
import json
import warnings
import os
import sys

warnings.filterwarnings('ignore')


In [2]:
import math

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)
  

In [12]:
filename = 'aoi.geojson'
outputFilename = 'tile-urls.json'
tileBaseUrl = "https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/%s/%s/%s.png"
zoomLevel = 8
downloadFolder = './training'

In [13]:
gj = gpd.read_file(filename)
gj.head()

,geometry
0,"POLYGON ((-111.40137 62.99516, -107.27051 62.9..."


In [14]:
minLat = gj.head().bounds.miny[0]
maxLat = gj.head().bounds.maxy[0]
minLon = gj.head().bounds.minx[0]
maxLon = gj.head().bounds.maxx[0]
maxLat

64.69910544204765

In [15]:
topLeftTile = deg2num(minLat,minLon,zoomLevel)
topLeftTile

(48, 69)

In [16]:
bottomRightTile = deg2num(maxLat,maxLon,zoomLevel)
bottomRightTile

(51, 67)

In [21]:
resultUrls = {}

result = ""
for x in range(topLeftTile[0], bottomRightTile[0] + 1):
    for y in range(bottomRightTile[1], topLeftTile[1] + 1):
        result = tileBaseUrl%(zoomLevel, y, x)
        resultUrls["%s_%s_%s.png"%(zoomLevel, y, x)] = result
        
resultUrls


{'8_67_48.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/8/67/48.png',
 '8_68_48.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/8/68/48.png',
 '8_69_48.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/8/69/48.png',
 '8_67_49.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/8/67/49.png',
 '8_68_49.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/8/68/49.png',
 '8_69_49.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_5

In [22]:
import json

with open(outputFilename, 'w') as outfile:
    json.dump(resultUrls, outfile)

In [25]:
import shutil

# clear the target folder
#if os.path.exists(downloadFolder) and os.path.isdir(downloadFolder):
#    shutil.rmtree(downloadFolder)

for root, dirs, files in os.walk(downloadFolder):
    for f in files:
        os.unlink(os.path.join(root, f))

# re-create the target folder
if not os.path.exists(downloadFolder):
    os.makedirs(downloadFolder)

'start to download %s tiles'%(len(resultUrls))


'start to download 12 tiles'

In [26]:
import requests
    
for fileName, targetUrl in resultUrls.items():
    r = requests.get(targetUrl, allow_redirects=True)
    open(downloadFolder + '/' + fileName, 'wb').write(r.content)
    
'downloaded %s tiles to %s'%(len(resultUrls), downloadFolder)

'downloaded 12 tiles to ./training'